In [1]:
%pylab inline

import copy

import numpy as np
import pandas as pd
import sys
import os
import re

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, RMSprop
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import TimeDistributed
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM, GRU

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.


In [2]:
path = "../data/aclimdb/"

ff = [path + "train/pos/" + x for x in os.listdir(path + "train/pos")] + \
     [path + "train/neg/" + x for x in os.listdir(path + "train/neg")] + \
     [path + "test/pos/" + x for x in os.listdir(path + "test/pos")] + \
     [path + "test/neg/" + x for x in os.listdir(path + "test/neg")]

TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)
    
input_label = ([1] * 12500 + [0] * 12500) * 2
input_text  = []

for f in ff:
    with open(f) as fin:
        pass
        input_text += [remove_tags(" ".join(fin.readlines()))]

In [3]:
num_words = 5000
max_len = 400
tok = Tokenizer(num_words)
tok.fit_on_texts(input_text[:25000])

In [11]:
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tok, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [4]:
X_train = tok.texts_to_sequences(input_text[:25000])
X_test  = tok.texts_to_sequences(input_text[25000:])
y_train = input_label[:25000]
y_test  = input_label[25000:]

X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test  = sequence.pad_sequences(X_test,  maxlen=max_len)

In [5]:
model = Sequential()

model.add(Embedding(num_words, 300, input_length=max_len))
model.add(Dropout(0.5))

model.add(GRU(16,activation='relu'))

model.add(Dense(128))
model.add(Dropout(0.5))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [6]:
model.fit(X_train, y_train, batch_size=32, nb_epoch=5, verbose=1,
          validation_data=(X_test, y_test))

/opt/conda/lib/python3.6/site-packages/keras/models.py:981: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 25000 samples, validate on 25000 samples
Epoch 1/5
25000/25000 [==============================] - 274s 11ms/step - loss: 0.5322 - acc: 0.7294 - val_loss: 0.2926 - val_acc: 0.8758
Epoch 2/5
25000/25000 [==============================] - 268s 11ms/step - loss: 0.2687 - acc: 0.8914 - val_loss: 0.2852 - val_acc: 0.8838
Epoch 3/5
25000/25000 [==============================] - 263s 11ms/step - loss: 0.2152 - acc: 0.9174 - val_loss: 0.2458 - val_acc: 0.8998
Epoch 4/5
25000/25000 [==============================] - 259s 10ms/step - loss: 0.1839 - acc: 0.9304 - val_loss: 0.3089 - val_acc: 0.8860
Epoch 5/5
25000/25000 [==============================] - 255s 10ms/step - loss: 0.1628 - acc: 0.9394 - val_loss: 0.2572 - val_acc: 0.8988


In [7]:
from keras.models import load_model

model.save('model_imdb.h5') 